In [149]:
#vårt projekt

from typing import Tuple

from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
from torch.utils.data import dataset
from torch.nn.utils.rnn import pad_sequence

import random
import numpy as np
import math

import matplotlib.pyplot as plt
from tqdm import tqdm

In [150]:
#pip install sentencepiece

In [151]:
import sentencepiece as spm

In [152]:
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)

In [153]:
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
DEVICE = torch.device(device)
DEVICE


device(type='mps')

In [154]:
#Alice paths
data_path = '/Users/alicetottie/Downloads/projekt_data/train_paracrawl.en'
data_path_2 = '/Users/alicetottie/Downloads/projekt_data/train_paracrawl.sv'

#Maja paths
# data_path = '/Users/majarygard/Documents/LTH/Projekt i Data/train_paracrawl.en'
# data_path_2 = '/Users/majarygard/Documents/LTH/Projekt i Data/train_paracrawl.sv'

#Pernilla paths
#data_path = 'C:\\Users\\nilla\\Plugg\\project-data\\train_paracrawl.en\\train_paracrawl.en'
#data_path_2 = 'C:\\Users\\nilla\\Plugg\\project-data\\train_paracrawl.sv\\train_paracrawl.sv'


In [155]:
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

with open(data_path_2, "r", encoding="utf-8") as f:
    lines2 = f.read().split("\n")    

In [156]:
len(lines)



4960283

In [157]:
len(lines2)

4960283

In [158]:
num_samples = 1000
input_texts = []
target_texts = []

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text= line.split("\t")
    input_texts.append(input_text)

with open("input_texts.txt", "w") as f:
    for input_text in input_texts:
        f.write('\t'.join(input_text) + '\n')
    
for line in lines2[: min(num_samples, len(lines2) - 1)]:
    target_text= line.split("\t")
    target_texts.append(target_text)  

with open("target_texts.txt", "w") as f:
    for target_text in target_texts:
        f.write('\t'.join(target_text) + '\n') 

In [159]:
input_texts[500]



['" This is my third course and I bought this concept a long time ago.']

In [160]:
target_texts[500]

['" Det här är min tredje kurs och jag har köpt det hela för länge sedan.']

In [161]:
TRAIN_PERCENTAGE = 0.8
train_val = int(TRAIN_PERCENTAGE * num_samples)
train_val

800

In [162]:
text_pairs = list(zip(input_texts, target_texts))
random.shuffle(text_pairs)
input_texts, target_texts = zip(*text_pairs)
input_texts, target_texts = list(input_texts), list(target_texts)


In [163]:
train_input_texts = input_texts[:train_val]
train_target_texts = target_texts[:train_val]

val_input_texts = input_texts[train_val:]
val_target_texts = target_texts[train_val:]

In [164]:
spm.SentencePieceTrainer.train('--input=input_texts.txt --model_prefix=m --vocab_size=2000 --model_type=bpe --pad_id=3')

# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')

# encode: text => id
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

# decode: id => text
print(sp.decode_pieces(['▁This', '▁is', '▁a', '▁t', 'est']))
print(sp.decode_ids([664, 63, 4, 3, 133]))

['▁This', '▁is', '▁a', '▁t', 'est']
[665, 64, 5, 4, 134]
This is a test
otherent tra


sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=input_texts.txt --model_prefix=m --vocab_size=2000 --model_type=bpe --pad_id=3
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: input_texts.txt
  input_format: 
  model_prefix: m
  model_type: BPE
  vocab_size: 2000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id:

In [245]:
def text2codes(texts, sp):
    codes = []
    for text in texts:
        text_l = str('<s>' + str(sp.encode_as_pieces(text)) + '</s>')
        print(text_l)
        code= sp.encode_as_ids(text_l)
        print(code)
        codes+= torch.tensor(code)
    return codes


In [246]:
train_input_texts[:3], train_target_texts[:3]

([["!!TheM!Z!! hasn't added any friends yet!"],
  ['" Memory / Summerday" gouache/wood/butterfly 48x75 / 48x92cm 2006'],
  ['" child safety in Egypt "" harmfulness of the sun in Egypt on baby skin "" what medications the child in Egypt "" where on holiday with his family to Egypt "" pediatrician "" in which the child free hotels "']],
 [['!!TheM!Z!! har inte lagt till några vänner ännu!'],
  ['"Hågkomst / Sommardag" gouache/trä 48x75 /48x92cm2006'],
  ['" barnsäkerhet i Egypten "" skadlighet solen i Egypten på baby hud "" vilka mediciner barnet i Egypten "" där på semester med sin familj till Egypten "" barnläkare "" där barnet hotellsökningen "']])

In [247]:
text2codes(train_input_texts, sp)[:3]

<s>[['▁!!', 'The', 'M', '!', 'Z', '!!', '▁hasn', "'", 't', '▁added', '▁any', '▁friends', '▁yet', '!']]</s>
[1898, 0, 1905, 1992, 1981, 1981, 1945, 314, 1945, 1921, 606, 888, 1945, 1921, 606, 1928, 1945, 1921, 606, 1938, 1945, 1921, 606, 1976, 1945, 1921, 606, 460, 1945, 1921, 606, 342, 1945, 1921, 909, 714, 606, 1900, 1945, 1921, 606, 1785, 1945, 1921, 606, 242, 1945, 1921, 606, 1041, 1945, 1921, 606, 304, 1945, 1921, 606, 1938, 1945, 1982, 1982, 0, 1966, 1905, 1992]
<s>[['▁"', '▁M', 'em', 'ory', '▁/', '▁S', 'um', 'mer', 'day', '"', '▁g', 'ou', 'ac', 'he', '/', 'wood', '/', 'but', 'ter', 'f', 'ly', '▁4', '8', 'x', '7', '5', '▁/', '▁4', '8', 'x', '92', 'cm', '▁200', '6']]</s>
[1898, 0, 1905, 1992, 1981, 1981, 1945, 909, 1921, 606, 67, 1945, 1921, 606, 171, 1945, 1921, 606, 354, 1945, 1921, 606, 464, 1945, 1921, 606, 50, 1945, 1921, 606, 130, 1945, 1921, 606, 498, 1945, 1921, 606, 894, 1945, 1921, 606, 1911, 1945, 1921, 606, 54, 1945, 1921, 606, 27, 1945, 1921, 606, 71, 1945, 1921, 606, 

[tensor(1898), tensor(0), tensor(1905)]

In [207]:
def codes2text(codes):
    texts = []
    for code in codes:
        code_l = list(code)
        texts += [list(map(lambda x: sp.id_to_piece.get(x.item(), 0), code_l))]
    return texts

In [169]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super().__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)
                        * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)
        self.emb_size = emb_size

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding * math.sqrt(self.emb_size)
                            + self.pos_embedding[:token_embedding.size(0), :])

In [170]:
pe = PositionalEncoding(10, 0.1)

In [171]:
torch.zeros(1, 5, 10)



tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [172]:
pe(torch.zeros(1, 5, 10))

tensor([[[0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 1.1111,
          0.0000, 0.0000],
         [0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 1.1111,
          0.0000, 1.1111],
         [0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 1.1111,
          0.0000, 1.1111],
         [0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 0.0000, 0.0000, 1.1111,
          0.0000, 1.1111],
         [0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 1.1111,
          0.0000, 1.1111]]])